In [14]:
"""In this document the steps include:
1) Importing the data and then shrinking it (based on Jon's models)
2) Categorizing the items into categories and also ratings per user for each category
3) Then grouping the data by reviewers and by items 
4) Predicting based on the rating and how much of each category a user has bought
"""

import pandas as pd
import gzip
import d2l
import math
import time
import pandas as pd
import random
import numpy as np

In [2]:
# Import Select Columns
count=0
def readGz(f):
  for l in gzip.open(f):
        yield eval(l)


In [3]:
train_users = []
for l in readGz("train.json.gz"):
    train_users.append(l['reviewerID'])
    
item = []
for l in readGz("train.json.gz"):
    item.append(l['itemID'])
    
categories = []
# for l in readGz("/content/drive/My Drive/train.json.gz"):
for l in readGz("train.json.gz"):
    categories.append(l['categoryID'])
    
helpful = []
# for l in readGz("/content/drive/My Drive/train.json.gz"):
for l in readGz("train.json.gz"):
    helpful.append(l['helpful'])
    
rating = []
# for l in readGz("/content/drive/My Drive/train.json.gz"):
for l in readGz("train.json.gz"):
    rating.append(l['rating'])

price=[]
for l in readGz("train.json.gz"):
    if 'price' in l.keys():
        price.append(l['price'])
    else:
        price.append(0)

In [10]:

#Import Test data

test_allcol = pd.read_csv('pairs_Purchase.csv')

pairings = list(test_allcol['reviewerID-itemID'])
test_users = []
test_items = []

for pairing in pairings:
    u, i = pairing.split('-')
    test_users.append(u)
    test_items.append(i)

In [12]:

# Convert to Pandas df
train_df = pd.DataFrame(train_users, columns = ['ReviewerID'])
test_df = pd.DataFrame(test_users, columns = ['ReviewerID'])

train_df['ItemID'] = pd.DataFrame(item)
test_df['ItemID'] = pd.DataFrame(item)


#train_df['ItemID'] = pd.DataFrame(item, columns = ['ItemID'])
train_df['Rating'] = pd.DataFrame(rating, columns = ['Rating'])
train_df['Price']=pd.DataFrame(price, columns = ['Price'])

train_df

,ReviewerID,ItemID,Rating,Price
0,U490934656,I402344648,4.0,0.00
1,U714157797,I697650540,4.0,0.00
2,U507366950,I464613034,5.0,19.99
3,U307862152,I559560885,2.0,18.99
4,U742726598,I476005312,5.0,0.00
5,U659368900,I342631873,4.0,0.00
6,U361187730,I773829721,5.0,130.00
7,U749984810,I740935639,5.0,16.55
8,U352382050,I570436467,5.0,0.00
9,U536912400,I025792869,5.0,54.99


In [16]:
#Adding category (based on Jon's code)
train_df['categoryID'] = pd.DataFrame(categories, columns = ['categoryID'])
train_df['Cat0'] = np.where(train_df['categoryID'] == 0, 1, 0)
train_df['Cat1'] = np.where(train_df['categoryID'] == 1, 1, 0)
train_df['Cat2'] = np.where(train_df['categoryID'] == 2, 1, 0)
train_df['Cat3'] = np.where(train_df['categoryID'] == 3, 1, 0)
train_df['Cat4'] = np.where(train_df['categoryID'] == 4, 1, 0)
train_df.head()


,ReviewerID,ItemID,Rating,Price,categoryID,Cat0,Cat1,Cat2,Cat3,Cat4
0,U490934656,I402344648,4.0,0.00,0,1,0,0,0,0
1,U714157797,I697650540,4.0,0.00,0,1,0,0,0,0
2,U507366950,I464613034,5.0,19.99,0,1,0,0,0,0
3,U307862152,I559560885,2.0,18.99,0,1,0,0,0,0
4,U742726598,I476005312,5.0,0.00,0,1,0,0,0,0


In [33]:
#Adding rating per category. This will later be averaged per user
train_df['Cat0Rating'] = np.where(train_df['categoryID'] == 0, train_df['Rating'], 0)
train_df['Cat1Rating'] = np.where(train_df['categoryID'] == 1, train_df['Rating'], 0)
train_df['Cat2Rating'] = np.where(train_df['categoryID'] == 2, train_df['Rating'], 0)
train_df['Cat3Rating'] = np.where(train_df['categoryID'] == 3, train_df['Rating'], 0)
train_df['Cat4Rating'] = np.where(train_df['categoryID'] == 4, train_df['Rating'], 0)

In [34]:
test_df


,ReviewerID,ItemID,TestReviewCount
0,U938994110,I402344648,1
1,U181459539,I697650540,1
2,U941668816,I464613034,1
3,U768449391,I559560885,1
4,U640450168,I476005312,1
5,U087574132,I342631873,1
6,U885457860,I773829721,1
7,U319023404,I740935639,1
8,U535965656,I570436467,1
9,U883645154,I025792869,1


In [35]:
# Get distinct list of test reviewers
test_df['TestReviewCount'] = 1

test_user_counts = test_df.groupby(['ReviewerID']).sum()['TestReviewCount']
test_user_counts = test_user_counts.to_frame()
test_user_counts.head()

,TestReviewCount
ReviewerID,
U000005569,1
U000089279,1
U000132800,1
U000198945,2
U000243198,2


In [36]:
type(test_user_counts)

pandas.core.frame.DataFrame

In [37]:

train_df_shrink = pd.merge(train_df, test_user_counts, how = 'inner', 
                           on = ['ReviewerID'], suffixes = ('Train', 'Test'))
train_df_shrink.head()

,ReviewerID,ItemID,Rating,Price,categoryID,Cat0,Cat1,Cat2,Cat3,Cat4,Cat0Rating,Cat1Rating,Cat2Rating,Cat3Rating,Cat4Rating,TestReviewCount
0,U490934656,I402344648,4.0,0.00,0,1,0,0,0,0,4.0,0.0,0.0,0.0,0.0,1
1,U490934656,I330290793,3.0,0.00,0,1,0,0,0,0,3.0,0.0,0.0,0.0,0.0,1
2,U490934656,I296399509,5.0,6.74,0,1,0,0,0,0,5.0,0.0,0.0,0.0,0.0,1
3,U361187730,I773829721,5.0,130.00,0,1,0,0,0,0,5.0,0.0,0.0,0.0,0.0,1
4,U361187730,I781019543,5.0,0.00,1,0,1,0,0,0,0.0,5.0,0.0,0.0,0.0,1


In [38]:
train_df_shrink.to_csv('train_reduced.csv', index = False)

In [39]:
test_user_counts

,TestReviewCount
ReviewerID,
U000005569,1
U000089279,1
U000132800,1
U000198945,2
U000243198,2
U000446326,1
U000538954,1
U000570833,1
U000625341,3


In [40]:
train_df_shrink

,ReviewerID,ItemID,Rating,Price,categoryID,Cat0,Cat1,Cat2,Cat3,Cat4,Cat0Rating,Cat1Rating,Cat2Rating,Cat3Rating,Cat4Rating,TestReviewCount
0,U490934656,I402344648,4.0,0.00,0,1,0,0,0,0,4.0,0.0,0.0,0.0,0.0,1
1,U490934656,I330290793,3.0,0.00,0,1,0,0,0,0,3.0,0.0,0.0,0.0,0.0,1
2,U490934656,I296399509,5.0,6.74,0,1,0,0,0,0,5.0,0.0,0.0,0.0,0.0,1
3,U361187730,I773829721,5.0,130.00,0,1,0,0,0,0,5.0,0.0,0.0,0.0,0.0,1
4,U361187730,I781019543,5.0,0.00,1,0,1,0,0,0,0.0,5.0,0.0,0.0,0.0,1
5,U361187730,I373694144,5.0,19.95,0,1,0,0,0,0,5.0,0.0,0.0,0.0,0.0,1
6,U361187730,I057254930,5.0,13.20,1,0,1,0,0,0,0.0,5.0,0.0,0.0,0.0,1
7,U749984810,I740935639,5.0,16.55,0,1,0,0,0,0,5.0,0.0,0.0,0.0,0.0,1
8,U749984810,I623236211,5.0,0.00,0,1,0,0,0,0,5.0,0.0,0.0,0.0,0.0,1
9,U749984810,I532670196,5.0,9.17,1,0,1,0,0,0,0.0,5.0,0.0,0.0,0.0,1


In [69]:
#Grouping per user to find the number of categories bought per user and also the sum of all the ratings (this will be averaged soon)
train_users_grouped = train_df_shrink.groupby('ReviewerID').sum()
train_users_grouped = train_users_grouped.drop(columns = ['categoryID'])
train_users_grouped = train_users_grouped.drop(columns = ['Rating'])
train_users_grouped.head()
#train_users_grouped['Rating']=pd.DataFrame(train_df_shrink, columns = ['Rating'])



,Price,Cat0,Cat1,Cat2,Cat3,Cat4,Cat0Rating,Cat1Rating,Cat2Rating,Cat3Rating,Cat4Rating,TestReviewCount
ReviewerID,,,,,,,,,,,,
U000005569,28.98,5,0,0,0,0,25.0,0.0,0.0,0.0,0.0,5
U000089279,0.00,0,1,0,1,0,0.0,5.0,0.0,5.0,0.0,2
U000132800,92.58,7,0,0,0,0,30.0,0.0,0.0,0.0,0.0,7
U000198945,158.35,3,0,0,0,0,12.0,0.0,0.0,0.0,0.0,6
U000243198,7.95,0,3,0,0,0,0.0,15.0,0.0,0.0,0.0,6


In [71]:
#Now grouping but taking the average, this way we can find the average rating the person gives per category
train_users_grouped['Cat0Rating'] = train_users_grouped['Cat0Rating']/train_users_grouped['Cat0']
train_users_grouped['Cat1Rating'] = train_users_grouped['Cat1Rating']/train_users_grouped['Cat1']
train_users_grouped['Cat2Rating'] = train_users_grouped['Cat2Rating']/train_users_grouped['Cat2']
train_users_grouped['Cat3Rating'] = train_users_grouped['Cat3Rating']/train_users_grouped['Cat3']
train_users_grouped['Cat4Rating'] = train_users_grouped['Cat4Rating']/train_users_grouped['Cat4']
train_users_grouped.head()

,Price,Cat0,Cat1,Cat2,Cat3,Cat4,Cat0Rating,Cat1Rating,Cat2Rating,Cat3Rating,Cat4Rating,TestReviewCount
ReviewerID,,,,,,,,,,,,
U000005569,28.98,5,0,0,0,0,5.000000,NaN,NaN,NaN,NaN,5
U000089279,0.00,0,1,0,1,0,NaN,5.0,NaN,5.0,NaN,2
U000132800,92.58,7,0,0,0,0,4.285714,NaN,NaN,NaN,NaN,7
U000198945,158.35,3,0,0,0,0,4.000000,NaN,NaN,NaN,NaN,6
U000243198,7.95,0,3,0,0,0,NaN,5.0,NaN,NaN,NaN,6


In [77]:
train_users_grouped.columns

Index(['Price', 'Cat0', 'Cat1', 'Cat2', 'Cat3', 'Cat4', 'Cat0Rating',
       'Cat1Rating', 'Cat2Rating', 'Cat3Rating', 'Cat4Rating',
       'TestReviewCount'],
      dtype='object')

In [78]:
train_users_grouped.loc['U000005569']['Cat0']

5.0

In [72]:
#Grouping by item and then finding the total item and getting the category for all items 
item_grouped = train_df_shrink.groupby('ItemID').max()
item_grouped.head()
item_cat=item_grouped['categoryID']

In [73]:
item_cat

ItemID
I000024906    0
I000030838    1
I000253607    1
I000310825    1
I000350237    0
I000430159    0
I000533741    0
I000622228    0
I000636940    1
I000647361    4
I000710737    0
I000791689    0
I000904797    0
I000928955    0
I001013648    1
I001213759    0
I001292472    0
I001512439    1
I001618316    0
I001745691    0
I001766836    0
I001775439    0
I001810459    0
I001829045    0
I001947008    0
I002037778    1
I002094053    2
I002099577    0
I002102908    1
I002210759    1
             ..
I998223345    0
I998278194    0
I998280023    0
I998280369    0
I998374414    0
I998470090    0
I998574059    1
I998593499    1
I998817694    0
I998823098    0
I998838167    0
I998939378    1
I999034678    1
I999124072    1
I999196749    0
I999198554    0
I999200131    0
I999207760    0
I999215823    0
I999329564    0
I999334844    2
I999418262    0
I999454484    0
I999503293    0
I999566818    0
I999583554    0
I999658909    0
I999715871    1
I999879552    0
I999917660    0
Name: categoryID,

In [96]:
#predictionFile='pairs_Purchase_Original2.csv'
#prediction = pd.read_csv(predictionFile)

In [108]:
#loading up the prediction file again with the reviewer and item split in different columns
predictionFile='pairs_Purchase_Original.csv'
prediction = pd.read_csv(predictionFile)

In [109]:
"""Here, will go over every row in the prediction column and make a prediction based on the heuristics 
of whether a person has 
1) Bought a category of something in the past 
and 2) they rated that category well!
"""
count=0
items=list(train_df_shrink['ItemID'].unique())
reviewers=list(train_df_shrink['ReviewerID'].unique())
for i in range(len(list(prediction['itemID']))):
    reviewID=str(prediction['reviewerID'][i])
    if prediction['itemID'][i] in items and reviewID in reviewers:
        categoryID='Cat'+str(item_cat[prediction['itemID'][i]])
        categorySum=train_users_grouped.loc[reviewID][categoryID]
        categoryRatingText='Cat'+str(item_cat[prediction['itemID'][i]])+'Rating'
        CategoryRating=train_users_grouped.loc[reviewID][categoryRatingText]
        print('Rating',CategoryRating)
    else:
        categoryID='Cat0'
        categorySum=0
        CategoryRating=0
    print(categoryID)
    #print(reviewID)
    #index=train_users_grouped.index[i]
    
    if categorySum>5 and CategoryRating>4:
        prediction['prediction'][i]=1
        print('True')
    else:
        print('False')
    count+=1
    print(count)

Rating 5.0
Cat0
False
1
Rating 5.0
Cat1
False
2
Rating 3.0
Cat0
False
3
Rating 4.466666666666667
Cat0
True
4
Rating 4.2727272727272725
Cat0
True
5
Cat0
False
6
Rating nan
Cat1
False
7
Rating 4.6
Cat0
False
8
Rating 4.333333333333333
Cat1
False
9
Rating 4.5
Cat1


C:\Users\anmalpan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


True
10
Rating nan
Cat3
False
11
Rating 4.666666666666667
Cat0
False
12
Rating 3.5
Cat0
False
13
Rating 2.3333333333333335
Cat0
False
14
Rating 4.0
Cat1
False
15
Rating 4.25
Cat1
False
16
Rating 5.0
Cat0
False
17
Rating 4.0
Cat0
False
18
Rating nan
Cat1
False
19
Rating 4.25
Cat0
True
20
Rating 4.615384615384615
Cat0
True
21
Cat0
False
22
Rating nan
Cat4
False
23
Rating nan
Cat1
False
24
Rating 3.0
Cat1
False
25
Rating 4.0
Cat0
False
26
Rating nan
Cat0
False
27
Rating 3.8
Cat0
False
28
Rating 3.7142857142857144
Cat0
False
29
Rating 3.25
Cat0
False
30
Rating 4.5
Cat0
False
31
Rating 4.0
Cat0
False
32
Rating 4.333333333333333
Cat0
False
33
Rating 3.0
Cat0
False
34
Rating nan
Cat0
False
35
Rating nan
Cat1
False
36
Rating 3.5
Cat0
False
37
Rating nan
Cat0
False
38
Rating 4.666666666666667
Cat1
False
39
Rating 3.5
Cat0
False
40
Rating 5.0
Cat1
False
41
Rating 5.0
Cat0
False
42
Rating 5.0
Cat1
False
43
Rating 3.3333333333333335
Cat1
False
44
Rating 4.6
Cat0
False
45
Rating 4.916666666666667
C

False
313
Rating 2.0
Cat0
False
314
Rating 4.666666666666667
Cat0
False
315
Rating 4.375
Cat1
True
316
Rating 4.0
Cat0
False
317
Rating 5.0
Cat1
False
318
Rating 1.0
Cat0
False
319
Cat0
False
320
Rating 3.0
Cat0
False
321
Rating 5.0
Cat0
False
322
Rating 3.0
Cat0
False
323
Rating 4.0
Cat1
False
324
Rating 4.5
Cat0
False
325
Cat0
False
326
Cat0
False
327
Rating 5.0
Cat0
False
328
Rating 4.333333333333333
Cat0
False
329
Rating nan
Cat1
False
330
Rating 2.6666666666666665
Cat0
False
331
Rating 4.666666666666667
Cat1
False
332
Rating nan
Cat0
False
333
Rating 5.0
Cat1
False
334
Rating 4.5
Cat0
False
335
Rating 2.0
Cat0
False
336
Rating 5.0
Cat1
False
337
Rating 5.0
Cat0
False
338
Rating 4.5
Cat0
True
339
Rating 4.076923076923077
Cat0
True
340
Rating 4.25
Cat1
False
341
Rating 5.0
Cat1
False
342
Rating 3.6666666666666665
Cat0
False
343
Rating 4.666666666666667
Cat0
False
344
Rating 3.0
Cat0
False
345
Rating nan
Cat0
False
346
Rating 4.333333333333333
Cat0
False
347
Rating 4.8
Cat0
False
348

Rating 4.6
Cat0
False
610
Rating 4.0
Cat1
False
611
Rating 4.9
Cat0
True
612
Rating 4.428571428571429
Cat0
True
613
Rating nan
Cat1
False
614
Rating 5.0
Cat0
False
615
Rating 3.2857142857142856
Cat0
False
616
Rating 4.666666666666667
Cat0
False
617
Cat0
False
618
Rating 4.7
Cat1
True
619
Rating 4.75
Cat0
False
620
Rating 2.75
Cat0
False
621
Rating 3.75
Cat0
False
622
Rating 4.25
Cat1
False
623
Rating 4.0
Cat1
False
624
Rating 5.0
Cat1
False
625
Rating 5.0
Cat0
False
626
Rating 3.0
Cat0
False
627
Rating 4.833333333333333
Cat0
True
628
Rating 3.5
Cat0
False
629
Rating nan
Cat0
False
630
Rating 3.5
Cat0
False
631
Rating 4.0
Cat0
False
632
Rating 5.0
Cat0
True
633
Rating 4.0
Cat0
False
634
Rating 5.0
Cat0
False
635
Rating 5.0
Cat0
False
636
Rating 4.0
Cat0
False
637
Rating 3.6666666666666665
Cat1
False
638
Rating 3.4
Cat0
False
639
Rating 5.0
Cat1
False
640
Rating 4.0
Cat0
False
641
Rating 2.8333333333333335
Cat0
False
642
Rating 4.666666666666667
Cat0
True
643
Rating nan
Cat4
False
644
Ra

...

26643
Rating 4.333333333333333
Cat0
False
26644
Rating 3.2
Cat0
False
26645
Rating nan
Cat1
False
26646
Rating 3.3333333333333335
Cat0
False
26647
Rating 2.75
Cat0
False
26648
Rating nan
Cat2
False
26649
Rating 3.0
Cat0
False
26650
Rating 4.6
Cat0
False
26651
Rating 4.8
Cat0
False
26652
Rating 4.166666666666667
Cat0
True
26653
Rating 5.0
Cat0
False
26654
Rating 3.8
Cat0
False
26655
Rating 3.0
Cat0
False
26656
Rating 2.75
Cat0
False
26657
Rating 5.0
Cat0
False
26658
Rating 4.4
Cat0
False
26659
Rating 5.0
Cat1
False
26660
Rating 3.7777777777777777
Cat0
False
26661
Rating nan
Cat1
False
26662
Rating 5.0
Cat0
False
26663
Rating 4.0
Cat0
False
26664
Rating 5.0
Cat0
False
26665
Rating nan
Cat1
False
26666
Rating 4.333333333333333
Cat0
False
26667
Rating 4.5
Cat0
True
26668
Rating 4.0
Cat1
False
26669
Rating 3.4
Cat0
False
26670
Rating 4.5
Cat0
False
26671
Rating 4.666666666666667
Cat0
False
26672
Rating nan
Cat1
False
26673
Rating 4.0
Cat0
False
26674
Rating 5.0
Cat1
False
26675
Rating 5.0
C

Cat1
False
26946
Rating 3.0
Cat0
False
26947
Rating 5.0
Cat4
False
26948
Rating 5.0
Cat1
False
26949
Cat0
False
26950
Rating 5.0
Cat0
False
26951
Rating nan
Cat0
False
26952
Rating 4.0
Cat0
False
26953
Rating 2.75
Cat0
False
26954
Rating 4.8
Cat0
False
26955
Rating 3.6666666666666665
Cat1
False
26956
Rating 4.6
Cat0
False
26957
Rating 3.8333333333333335
Cat0
False
26958
Rating 4.4
Cat0
False
26959
Rating nan
Cat0
False
26960
Rating nan
Cat1
False
26961
Rating nan
Cat1
False
26962
Rating nan
Cat1
False
26963
Rating 5.0
Cat0
False
26964
Rating 4.5
Cat0
False
26965
Rating 3.4285714285714284
Cat0
False
26966
Rating 4.2
Cat0
False
26967
Rating 5.0
Cat1
False
26968
Cat0
False
26969
Rating nan
Cat0
False
26970
Cat0
False
26971
Rating 3.8
Cat0
False
26972
Rating nan
Cat1
False
26973
Rating 4.5
Cat0
False
26974
Rating 5.0
Cat1
False
26975
Rating 4.4
Cat0
False
26976
Rating 3.9
Cat0
False
26977
Rating 5.0
Cat0
False
26978
Rating 5.0
Cat1
False
26979
Rating nan
Cat1
False
26980
Rating 2.666666666

False
27249
Rating 5.0
Cat0
False
27250
Rating nan
Cat0
False
27251
Rating nan
Cat1
False
27252
Rating nan
Cat1
False
27253
Rating 4.454545454545454
Cat0
True
27254
Rating 5.0
Cat1
False
27255
Rating 3.4166666666666665
Cat0
False
27256
Rating 5.0
Cat1
True
27257
Rating 4.6
Cat0
False
27258
Rating 4.0
Cat0
False
27259
Rating 4.636363636363637
Cat0
True
27260
Rating nan
Cat1
False
27261
Rating 2.3333333333333335
Cat0
False
27262
Rating 4.5
Cat1
True
27263
Rating 5.0
Cat1
False
27264
Rating 5.0
Cat1
False
27265
Rating 4.0
Cat0
False
27266
Rating nan
Cat1
False
27267
Rating 4.75
Cat0
False
27268
Rating 3.0
Cat0
False
27269
Rating 3.6
Cat0
False
27270
Rating 5.0
Cat0
False
27271
Rating 4.0
Cat0
False
27272
Rating 3.8666666666666667
Cat0
False
27273
Rating 4.0
Cat0
False
27274
Rating 5.0
Cat0
False
27275
Rating 4.142857142857143
Cat0
True
27276
Rating 1.25
Cat0
False
27277
Rating 4.5
Cat0
False
27278
Rating nan
Cat1
False
27279
Rating 4.333333333333333
Cat1
False
27280
Rating 4.8333333333333

False
27552
Rating 4.25
Cat0
False
27553
Rating 4.666666666666667
Cat0
True
27554
Rating 4.285714285714286
Cat0
True
27555
Rating nan
Cat1
False
27556
Rating 4.2
Cat0
False
27557
Rating 4.0
Cat1
False
27558
Rating 5.0
Cat0
False
27559
Rating 5.0
Cat0
False
27560
Rating nan
Cat0
False
27561
Rating 5.0
Cat0
False
27562
Rating 4.1
Cat0
True
27563
Rating 4.0
Cat0
False
27564
Rating nan
Cat0
False
27565
Rating 4.666666666666667
Cat0
False
27566
Rating 4.166666666666667
Cat0
True
27567
Rating 4.5
Cat0
False
27568
Rating 5.0
Cat1
False
27569
Rating 5.0
Cat0
False
27570
Rating 4.8
Cat0
False
27571
Rating 4.0
Cat0
False
27572
Rating 5.0
Cat1
False
27573
Rating 4.6
Cat0
False
27574
Rating 5.0
Cat0
False
27575
Rating 3.4
Cat0
False
27576
Rating 3.5714285714285716
Cat0
False
27577
Rating 4.333333333333333
Cat0
False
27578
Rating nan
Cat0
False
27579
Rating 4.75
Cat0
False
27580
Rating nan
Cat1
False
27581
Rating 4.6
Cat0
False
27582
Rating 4.2
Cat0
False
27583
Rating 3.4
Cat0
False
27584
Rating 4.

27855
Rating 3.6666666666666665
Cat0
False
27856
Rating nan
Cat0
False
27857
Rating 3.25
Cat0
False
27858
Rating 3.5714285714285716
Cat0
False
27859
Rating 5.0
Cat0
False
27860
Rating 4.333333333333333
Cat0
False
27861
Rating 4.0
Cat0
False
27862
Rating 5.0
Cat0
False
27863
Rating 4.0
Cat0
False
27864
Rating 5.0
Cat0
False
27865
Rating 4.75
Cat0
False
27866
Rating 4.0
Cat0
False
27867
Rating nan
Cat3
False
27868
Rating 5.0
Cat1
False
27869
Rating nan
Cat4
False
27870
Rating 5.0
Cat0
False
27871
Rating 4.666666666666667
Cat0
False
27872
Rating nan
Cat2
False
27873
Rating 4.666666666666667
Cat0
True
27874
Rating 5.0
Cat0
False
27875
Rating 3.5
Cat4
False
27876
Rating 4.5
Cat0
False
27877
Rating 5.0
Cat0
False
27878
Rating 4.6
Cat0
False
27879
Rating 3.8333333333333335
Cat0
False
27880
Rating 4.2
Cat0
False
27881
Rating 3.7058823529411766
Cat0
False
27882
Rating 3.0
Cat1
False
27883
Rating 3.7777777777777777
Cat0
False
27884
Rating nan
Cat0
False
27885
Rating nan
Cat0
False
27886
Rating 3

In [110]:
prediction

,reviewerID-itemID,reviewerID,itemID,prediction
0,U938994110-I529819131,U938994110,I529819131,0
1,U181459539-I863471064,U181459539,I863471064,1
2,U941668816-I684585522,U941668816,I684585522,0
3,U768449391-I782253949,U768449391,I782253949,1
4,U640450168-I232683472,U640450168,I232683472,1
5,U087574132-I014281144,U087574132,I014281144,0
6,U885457860-I600866492,U885457860,I600866492,0
7,U319023404-I050733439,U319023404,I050733439,1
8,U535965656-I929867818,U535965656,I929867818,0
9,U883645154-I158444048,U883645154,I158444048,1


In [111]:
prediction.to_csv('PredictionsCategory&RatingV6', sep='\t')

In [112]:
prediction

,reviewerID-itemID,reviewerID,itemID,prediction
0,U938994110-I529819131,U938994110,I529819131,0
1,U181459539-I863471064,U181459539,I863471064,1
2,U941668816-I684585522,U941668816,I684585522,0
3,U768449391-I782253949,U768449391,I782253949,1
4,U640450168-I232683472,U640450168,I232683472,1
5,U087574132-I014281144,U087574132,I014281144,0
6,U885457860-I600866492,U885457860,I600866492,0
7,U319023404-I050733439,U319023404,I050733439,1
8,U535965656-I929867818,U535965656,I929867818,0
9,U883645154-I158444048,U883645154,I158444048,1


In [68]:
train_users_grouped

,Price,Cat0,Cat1,Cat2,Cat3,Cat4,Cat0Rating,Cat1Rating,Cat2Rating,Cat3Rating,Cat4Rating,TestReviewCount
ReviewerID,,,,,,,,,,,,
U000005569,28.98,5,0,0,0,0,1.000000,NaN,NaN,NaN,NaN,5
U000089279,0.00,0,1,0,1,0,NaN,5.000000,NaN,5.0,NaN,2
U000132800,92.58,7,0,0,0,0,0.612245,NaN,NaN,NaN,NaN,7
U000198945,158.35,3,0,0,0,0,1.333333,NaN,NaN,NaN,NaN,6
U000243198,7.95,0,3,0,0,0,NaN,1.666667,NaN,NaN,NaN,6
U000446326,16.38,4,0,0,0,0,1.000000,NaN,NaN,NaN,NaN,4
U000538954,0.00,3,0,0,0,0,1.555556,NaN,NaN,NaN,NaN,3
U000570833,0.00,3,0,0,0,0,1.222222,NaN,NaN,NaN,NaN,3
U000625341,0.00,3,0,0,0,0,1.444444,NaN,NaN,NaN,NaN,9


In [ ]:
U000089279